Uncomment and run the cell below if you are in a Google Colab environment. It will:
1. Mount google drive. You will be asked to authenticate and give permissions.
2. Change directory to google drive.
3. Make a directory "hamilton-tutorials"
4. Change directory to it.
5. Clone this repository to your google drive
6. Move your current directory to the hello_world example
7. Install requirements.

This means that any modifications will be saved, and you won't lose them if you close your browser.

In [1]:
## 1. Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')
## 2. Change directory to google drive.
# %cd /content/drive/MyDrive
## 3. Make a directory "hamilton-tutorials"
# !mkdir hamilton-tutorials
## 4. Change directory to it.
# %cd hamilton-tutorials
## 5. Clone this repository to your google drive
# !git clone https://github.com/DAGWorks-Inc/hamilton/
## 6. Move your current directory to the hello_world example
# %cd hamilton/examples/hello_world
## 7. Install requirements.
# %pip install -r requirements.txt
# clear_output()  # optionally clear outputs
# To check your current working directory you can type `!pwd` in a cell and run it.

In [2]:
# Cell 2 - import modules to create part of the DAG from
# We use the autoreload extension that comes with ipython to automatically reload modules when
# the code in them changes.

# import the jupyter extension
%load_ext autoreload
# set it to only reload the modules imported
%autoreload 1

In [3]:
# Import modules
import pandas as pd
from dask import dataframe
from dask.distributed import (
    Client,
    LocalCluster,
)

from hamilton import (
    ad_hoc_utils,
    driver,
)
from hamilton.plugins import h_dask

In [4]:
# We'll place the data loaders into a new module

def spend(spend_location: str, spend_partitions: int) -> dataframe.Series:
    """Dummy function showing how to wire through loading data.

    :param spend_location:
    :param spend_partitions: number of partitions to segment the data into
    :return:
    """
    return dataframe.from_pandas(
        pd.Series([10, 10, 20, 40, 40, 50]), name="spend", npartitions=spend_partitions
    )


def signups(signups_location: str, signups_partitions: int) -> dataframe.Series:
    """Dummy function showing how to wire through loading data.

    :param signups_location:
    :param signups_partitions: number of partitions to segment the data into
    :return:
    """
    return dataframe.from_pandas(
        pd.Series([1, 10, 50, 100, 200, 400]), name="signups", npartitions=signups_partitions
    )

data_loaders = ad_hoc_utils.create_temporary_module(
    spend,
    signups,
    module_name="data_loaders",
)

In [5]:
# We'll place the spend calculations into a new module

def avg_3wk_spend(spend: pd.Series) -> pd.Series:
    """Rolling 3 week average spend."""
    return spend.rolling(3).mean()


def spend_per_signup(spend: pd.Series, signups: pd.Series) -> pd.Series:
    """The cost per signup in relation to spend."""
    return spend / signups


def spend_mean(spend: pd.Series) -> float:
    """Shows function creating a scalar. In this case it computes the mean of the entire column."""
    return spend.mean()


def spend_zero_mean(spend: pd.Series, spend_mean: float) -> pd.Series:
    """Shows function that takes a scalar. In this case to zero mean spend."""
    return spend - spend_mean


def spend_std_dev(spend: pd.Series) -> float:
    """Function that computes the standard deviation of the spend column."""
    return spend.std()


def spend_zero_mean_unit_variance(spend_zero_mean: pd.Series, spend_std_dev: float) -> pd.Series:
    """Function showing one way to make spend have zero mean and unit variance."""
    return spend_zero_mean / spend_std_dev

spend_calculations = ad_hoc_utils.create_temporary_module(
    avg_3wk_spend,
    spend_per_signup,
    spend_mean,
    spend_zero_mean,
    spend_std_dev,
    spend_zero_mean_unit_variance,
    module_name="spend_calculations",
)

In [6]:
# Set up the driver, input and output columns

cluster = LocalCluster()
client  = Client(cluster)

print(client.cluster)

adapter = h_dask.DaskGraphAdapter(
    client,
    h_dask.DaskDataFrameResult(),
    visualize_kwargs={"filename": "run_dask.png", "format": "png"},
    use_delayed=False,
    compute_at_end=False,
)

config = {
    "spend_location": "some file path",
    "spend_partitions": 2,
    "signups_location": "some file path",
    "signups_partitions": 2,
    "foobar": "some_other_data",
}

dr = driver.Driver(config, spend_calculations, data_loaders, adapter=adapter)

Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over time. See https://github.com/dagworks-inc/hamilton#usage-analytics--data-privacy for details.


LocalCluster(c13c9b83, 'tcp://127.0.0.1:56333', workers=4, threads=8, memory=16.00 GiB)


In [7]:
# Execute the driver.

output_columns = [
    "spend",
    "signups",
    "avg_3wk_spend",
    "spend_per_signup",
    "spend_mean",
    "spend_zero_mean_unit_variance",
    "foobar",
]

dask_df = dr.execute(output_columns)  # it's dask dataframe -- it hasn't been evaluated yet.
df = dask_df.compute()

# To visualize do `pip install "sf-hamilton[visualization]"` if you want these to work
dr.visualize_execution(output_columns, './hello_world_dask', {"format": "png"})
dr.display_all_functions("./my_full_dag.dot")
print(df.to_string())
client.shutdown()



   spend  signups  avg_3wk_spend  spend_per_signup  spend_mean  spend_zero_mean_unit_variance           foobar
0     10        1            NaN            10.000   28.333333                      -1.064405  some_other_data
1     10       10            NaN             1.000   28.333333                      -1.064405  some_other_data
2     20       50      13.333333             0.400   28.333333                      -0.483821  some_other_data
3     40      100      23.333333             0.400   28.333333                       0.677349  some_other_data
4     40      200      33.333333             0.200   28.333333                       0.677349  some_other_data
5     50      400      43.333333             0.125   28.333333                       1.257934  some_other_data
